# Earth Engine Data Retrieval Test
This notebook is a playground for retrieving data sets.

In [18]:
import ee
from PIL import ImageTk
from IPython.display import Image

ImportError: No module named _tkinter, please install the python-tk package

In [9]:
ee.Initialize()

In [10]:
nex = ee.ImageCollection("NASA/NEX-DCP30_ENSEMBLE_STATS")      # pull data from the nasa/nex-dcp30 climate collection
filtered = nex.filterDate('2010-01-01', '2018-07-05')          # define filtered date range

nex_image = filtered.median()                                  # define the image mosaic from the filtered date range

point = ee.Geometry.Point(-118, 37)
california = point.buffer(50000000).bounds().getInfo()['coordinates']

In [11]:
def createTimeBand(image):
  """Add a time band to the specified image"""
  # scale milliseconds by a large constant
  return image.addBands(image.metadata('system:time_start').divide(1e18))

In [12]:
def createConstantBand(image):
  """Add a constant time band to the specified image"""
  return ee.Image(1).addBands(image)

## Data collection
We now load the input image collection - projected climate data

In [13]:
collection = (ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS')
             .filterDate(ee.Date('2006-01-01'), ee.Date('2099-01-01'))
             .filter(ee.Filter.eq('scenario', 'rcp85'))
             .map(createTimeBand)
             .map(createConstantBand)
             .select(['constant', 'system:time_start', 'pr_mean', 'tasmax_mean']))

# compute least squares regression coefficients
linearRegression = (collection.reduce(
  ee.Reducer.linearRegression(
    numX = 2,
    numY = 2
  ))
)

# Compute robust linear regression coefficients.
robustLinearRegression = (collection.reduce(
  ee.Reducer.robustLinearRegression(
    numX = 2,
    numY = 2
  ))
)

# The results are array images that must be flattened for display.
# These lists label the information along each axis of the arrays.
bandNames = [['constant', 'time'],       # 0-axis variation.
             ['precip', 'temp']]         # 1-axis variation.

# Flatten the array images to get multi-band images according to the labels.
lrImage = linearRegression.select(['coefficients']).arrayFlatten(bandNames)
rlrImage = robustLinearRegression.select(['coefficients']).arrayFlatten(bandNames)

Image(url = lrImage.getThumbUrl({'region': california}))

# print rlrImage.getInfo(), lrImage.getInfo()

print("OLS estimates:", lrImage.reduceRegion(
  reducer = ee.Reducer.first(),
  geometry = ee.Geometry.Point([-96.0, 41.0]),
  scale = 1000
))

print("Robust estimates:", rlrImage.reduceRegion(
  reducer = ee.Reducer.first(),
  geometry = ee.Geometry.Point([-96.0, 41.0]),
  scale = 1000
))

# Image('https://developers.google.com/earth-engine/images/Reducers_linearFit.png')

('OLS estimates:', <ee.dictionary.Dictionary object at 0x7f2a5c8cecd0>)
('Robust estimates:', <ee.dictionary.Dictionary object at 0x7f2a5c8cea90>)


## Precipitation Predict
we now predict precipitation.
Blue = increase
red = decrease

In [14]:
precp_collection = (ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS')
                      .filter(ee.Filter.eq('scenario', 'rcp85'))
                      .filterDate(ee.Date('2006-01-01'), ee.Date('2050-01-01'))
                      .map(createTimeBand)
                   )

linearFit = precp_collection.select(['system:time_start', 'pr_mean']).reduce(ee.Reducer.linearFit())

